In [1]:
import os

In [2]:
%pwd

'c:\\Users\\farde\\YouTube\\End to end ml project\\End-to-end-ML-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\farde\\YouTube\\End to end ml project\\End-to-end-ML-Project'

In [22]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [23]:
from mlproject.constants import *
from mlproject.utlis.common import read_yaml,create_directories

In [24]:
class ConfigurationManger:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH,
            schema_filepath=SCHEMA_FILE_PATH):
        
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config=self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [25]:
import os
import urllib.request as request
import zipfile
from mlproject import logger
from mlproject.utlis.common import get_size

In [26]:
class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config=config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers=request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} download with following info:\n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
    def extract_zip_file(self):
        """
        zip file path:str
        Extrats the zip file into the data directory
        Function returns None
        """
        unzip_path=self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,"r") as zip_ref:
            zip_ref.extractall(unzip_path)

In [27]:
try:
    config=ConfigurationManger()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-04-02 17:23:34,081:INFO:common:yaml file: config\config.yaml loaded sucessfully]
[2024-04-02 17:23:34,085:INFO:common:yaml file: params.yaml loaded sucessfully]
[2024-04-02 17:23:34,093:INFO:common:yaml file: schema.yaml loaded sucessfully]
[2024-04-02 17:23:34,096:INFO:common:created directory at: artifacts]
[2024-04-02 17:23:34,101:INFO:common:created directory at: artifacts/data_ingestion]
[2024-04-02 17:23:36,287:INFO:3245653082:artifacts/data_ingestion/data.zip download with following info:
Connection: close
Content-Length: 23331
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "1379187672a27d24a5df588746f6973b46c21df7c34a2a1caa084df5b66d3118"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 7430:3CB9B:399DA6:4C569B:660BEEF6
Accept-Ranges: bytes
Date: Tue, 02 Apr 2024 11:53:33 GMT
V